In [0]:
    ride_df=spark.read.format('parquet').options(inferSchema=True, header=True).load('/Volumes/true_catalog/bronze/kagle_dataset_raw/nyc-rideshare-raw-data/rideshare_data.parquet')

In [0]:
zone_df=spark.read.format('csv').options(inferSchema=True, header=True).load('/Volumes/true_catalog/bronze/kagle_dataset_raw/nyc-rideshare-raw-data/taxi_zone_lookup.csv')

In [0]:
ride_df.limit(3).display()
zone_df.limit(3).display()

In [0]:
from pyspark.sql.functions import col

trip_df=ride_df.alias('r').join(
    zone_df.alias('z'),
    col('r.pickup_location') == col('z.LocationID'),
    'inner'
).join(zone_df.alias('dz'),col('dz.LocationID')==col('r.dropoff_location'),'inner').select('business','pickup_location',col('z.Zone').alias('pickup_zone'),col('dz.Zone').alias('dropoff_zone'),'time_of_day','hour_of_day','week_of_year','month_of_year','rideshare_profit')

In [0]:
trip_df.limit(10).display()

In [0]:
trip_df.write.mode("overwrite").saveAsTable("true_catalog.gold.trips")

In [0]:
%sql 
select count(*) from true_catalog.gold.trips

In [0]:
from pyspark.sql.functions import spark_partition_id, countDistinct

display(
    trip_df
    .withColumn("partition_id", spark_partition_id())
    .select("partition_id")
    .agg(countDistinct("partition_id").alias("num_partitions"))
)

In [0]:
trip_df=trip_df.repartition(10)
trip_df.rdd.getNumPartitions()

In [0]:
trip_df.write.format("delta").mode("overwrite").save('/Volumes/true_catalog/gold/delta_files/trips')

In [0]:
trip_df.rdd.getNumPartitions()

In [0]:
trip_df.select('business').distinct().show()

In [0]:
trip_df.write.format("delta").mode("overwrite").partitionBy('business').save('/Volumes/true_catalog/gold/delta_files/trips')